# Transformação de tipo de arquivo

> Visualizar pastas da conta de armazenamento

In [0]:
dbutils.fs.ls('abfss://livros@recomendador.dfs.core.windows.net/')

In [0]:
dbutils.fs.ls('abfss://livros@recomendador.dfs.core.windows.net/inbound')

## De csv para delta

> Ler arquivos

In [0]:
df_goodreads = spark.read.options(delimiter=",", header=True).csv('abfss://livros@recomendador.dfs.core.windows.net/inbound/goodreads_100k_books.csv', inferSchema=True)
df_kindle = spark.read.options(delimiter=",", header=True).csv('abfss://livros@recomendador.dfs.core.windows.net/inbound/kindle_data_v2.csv', inferSchema=True)

> Visualizar dados

In [0]:
df_kindle.toPandas().head(2)

In [0]:
df_goodreads.toPandas().head(2)

> Ajustar dados provenientes do GoodReads

In [0]:
def trocar_virgula_por_pontoevirgula(caminho):
    contagem_linha = 0

    with open(caminho, 'r') as arquivo:
        for linha in arquivo:
            for palavra in linha:
                if palavra == ',' and contagem_linha < 2:
                    linha[palavra] = ';'
                    contagem_linha += 1
                break
            contagem_linha == 0
            for palavra in linha[::-1]:
                if palavra == ',' and contagem_linha < 10:
                    linha[palavra] = ';'
                    contagem_linha += 1
                break
            
    return arquivo

> Alterar formato dos arquivos de saída

In [0]:
df_goodreads.read.options(delimiter=';', header=True).csv(trocar_virgula_por_pontoevirgula('abfss://livros@recomendador.dfs.core.windows.net/inbound/goodreads_100k_books.csv'), inferSchema=True)
df_goodreads.write.format('delta').options(header=True).mode('overwrite').save('abfss://livros@recomendador.dfs.core.windows.net/bronze/goodreads')

In [0]:
df_kindle.write.format('delta').options(header=True).mode('overwrite').save('abfss://livros@recomendador.dfs.core.windows.net/bronze/kindle')

## Exclusão de variáveis

In [0]:
del df_goodreads, df_kindle